#Mount, install and load file

In [ ]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
!pip3 install gensim
!pip3 install keras
!pip3 install tensorflow

In [ ]:
import nltk
nltk.download('punkt')

import pprint
import re
from lxml import etree
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

from gensim.models import Word2Vec

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Choose the file to run
# https://drive.google.com/file/d/1ZxiH7Oh6iNm7qu338t9BL045eJ4h0xhR/view?usp=sharing
# id = '1-1-NxTeGh9HktAvcYMDLuCe0jJK6Cngb'  #Percentage change  Combined_FAANG_percentage_2.2.csv
id = '1-4wGVlhCObAoAM_DOLL3D4YsJhOb1ZSj'  #Binary Combined_FAANG_binary_previous.csv
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Combined_FAANG_binary_previous.csv')  
train_data = pd.read_csv('Combined_FAANG_binary_previous.csv', sep=',')

In [ ]:
print("Number of training examples {}".format(len(train_data)))

Number of training examples 2566858


# Data Preparation

In [ ]:
train_data.drop(train_data[train_data['Date'] <= '2018-07-20'].index, inplace = True)
train_data

,symbol,message,datetime,user,message_id,Date,Time,label
1,AAPL,aapl watching gap fill 169 20,2018-11-24 07:02:32,1665234.0,146068732.0,2018-11-24,07:02:32,1
3,AAPL,aapl,2020-01-27 07:07:03,1229493.0,191978042.0,2020-01-27,07:07:03,0
5,AAPL,aapl loads cash hand great service business lo...,2018-11-01 23:39:14,123291.0,143688765.0,2018-11-01,23:39:14,1
8,AAPL,qq became euphoric calls exp week aiming ath f...,2020-05-13 02:13:00,2250451.0,212222428.0,2020-05-13,02:13:00,0
9,AAPL,spy novices like davey day trader lose money s...,2020-06-24 11:12:09,543250.0,222404886.0,2020-06-24,11:12:09,0
...,...,...,...,...,...,...,...,...
2566843,NFLX,nflx last 3 months big green candles followed ...,2019-05-01 18:21:25,637003.0,162589986.0,2019-05-01,18:21:25,1
2566848,NFLX,nflx full move key support,2020-03-12 16:52:43,677915.0,199933357.0,2020-03-12,16:52:43,0
2566849,NFLX,spy spx nflx nvda virtually volume today absen...,2019-10-14 18:16:28,55818.0,180328889.0,2019-10-14,18:16:28,1
2566853,NFLX,nflx sister owns kinda thinking telling sell b...,2019-01-11 20:51:22,607557.0,150426203.0,2019-01-11,20:51:22,1


In [ ]:
#Execute to drop Neutral in percentage change labelling
# train_data.drop(train_data[train_data['label'] == 0].index, inplace = True)
# train_data["label"].replace({-1: 0}, inplace=True)

In [ ]:
train_data['label'].value_counts()

1    548894
0    472556
Name: label, dtype: int64

In [ ]:
train_data = train_data.sample(frac=1)
train_data

,symbol,message,datetime,user,message_id,Date,Time,label
1864211,GOOGL,spy goog weekly otm options fill 69,2019-06-20 14:50:22,1045552.0,168115763.0,2019-06-20,14:50:22,1
389246,AAPL,aapl china virus back manufacturing halt soon,2020-06-17 15:30:03,3339933.0,220699728.0,2020-06-17,15:30:03,0
565670,AAPL,apple unveils new macbook air mac mini ipad pr...,2018-10-30 17:33:04,7108.0,143290974.0,2018-10-30,17:33:04,1
1051006,AAPL,aapl buy done,2018-11-20 14:40:38,1546473.0,145766080.0,2018-11-20,14:40:38,0
893247,AAPL,amzn aapl everything dropping afterhours,2018-11-26 21:22:14,431459.0,146208883.0,2018-11-26,21:22:14,1
...,...,...,...,...,...,...,...,...
1050799,AAPL,amzn fb googl aapl nflx pox traders,2018-10-30 04:57:24,1511688.0,143213258.0,2018-10-30,04:57:24,1
353861,AAPL,aapl eps grown impressive 24 97 past year,2019-04-01 02:12:17,47688.0,159038325.0,2019-04-01,02:12:17,1
557321,AAPL,spy hey bulls spy 39 even get 50 returns aapl ...,2019-04-03 15:11:54,836502.0,159385061.0,2019-04-03,15:11:54,1
832333,AAPL,aapl nvda fb highs,2018-12-17 15:12:53,397769.0,148115565.0,2018-12-17,15:12:53,0


In [ ]:
# Drop the NA values
train_data = train_data.dropna(subset=['message']).reset_index()  
train_data

,index,symbol,message,datetime,user,message_id,Date,Time,label
0,1864211,GOOGL,spy goog weekly otm options fill 69,2019-06-20 14:50:22,1045552.0,168115763.0,2019-06-20,14:50:22,1
1,389246,AAPL,aapl china virus back manufacturing halt soon,2020-06-17 15:30:03,3339933.0,220699728.0,2020-06-17,15:30:03,0
2,565670,AAPL,apple unveils new macbook air mac mini ipad pr...,2018-10-30 17:33:04,7108.0,143290974.0,2018-10-30,17:33:04,1
3,1051006,AAPL,aapl buy done,2018-11-20 14:40:38,1546473.0,145766080.0,2018-11-20,14:40:38,0
4,893247,AAPL,amzn aapl everything dropping afterhours,2018-11-26 21:22:14,431459.0,146208883.0,2018-11-26,21:22:14,1
...,...,...,...,...,...,...,...,...,...
1021445,1050799,AAPL,amzn fb googl aapl nflx pox traders,2018-10-30 04:57:24,1511688.0,143213258.0,2018-10-30,04:57:24,1
1021446,353861,AAPL,aapl eps grown impressive 24 97 past year,2019-04-01 02:12:17,47688.0,159038325.0,2019-04-01,02:12:17,1
1021447,557321,AAPL,spy hey bulls spy 39 even get 50 returns aapl ...,2019-04-03 15:11:54,836502.0,159385061.0,2019-04-03,15:11:54,1
1021448,832333,AAPL,aapl nvda fb highs,2018-12-17 15:12:53,397769.0,148115565.0,2018-12-17,15:12:53,0


In [ ]:
# Tokenization
target_text = train_data['message'].values
sentences=[]
sentences=[word_tokenize(sentence) for sentence in target_text]

# Prints only 10 (tokenised) sentences
print(sentences[:10])

[['spy', 'goog', 'weekly', 'otm', 'options', 'fill', '69'], ['aapl', 'china', 'virus', 'back', 'manufacturing', 'halt', 'soon'], ['apple', 'unveils', 'new', 'macbook', 'air', 'mac', 'mini', 'ipad', 'pro', '39', 'need', 'know', 'aapl'], ['aapl', 'buy', 'done'], ['amzn', 'aapl', 'everything', 'dropping', 'afterhours'], ['amzn', 'swing', 'higher', 'still', 'time', 'buy'], ['aapl', 'good', 'nrws', 'markets', 'peak', 'deaths', 'day', 'virus', 'control', 'stimulus'], ['aapl', 'hey', 'look', 'two', 'five', 'year', 'charts', 'bb', 'seeing', 'I', 'seeing'], ['ibio', 'canceled', 'nflx', 'subscription', 'entertainment', 'comes', 'board'], ['aapl', 'say', 'wait', 'r', 'get', 'ar', 'like', 'xs', 'new', 'game', 'playing', 'future', 'developers', 'like', 'atvi']]


# FastText implementation

In [ ]:
from gensim.models import FastText
ft_sg_model = FastText(sentences, size=100, window=5, min_count=5, workers=4, sg=1)

In [ ]:
similar_words=ft_sg_model.wv.most_similar("aapl")
pprint.pprint(similar_words)

[('aapll', 0.8026167154312134),
 ('mflx', 0.7674264907836914),
 ('appl', 0.7521910071372986),
 ('fb', 0.7379411458969116),
 ('nflx', 0.7165732383728027),
 ('spy', 0.7053292989730835),
 ('craapl', 0.7022196054458618),
 ('spyy', 0.6937599182128906),
 ('amzn', 0.6766653060913086),
 ('smzn', 0.6752709150314331)]


Splitting data into 90:10 ratio

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_data['message'].values, train_data['label'].values, test_size=0.1, random_state=42)

In [ ]:
df_train = pd.DataFrame(y_train,columns=['label'])
df_train['label'].value_counts()

1    494148
0    425157
Name: label, dtype: int64

In [ ]:
df_test = pd.DataFrame(y_test,columns=['label'])
df_test['label'].value_counts()

1    54746
0    47399
Name: label, dtype: int64

In [ ]:
import os
import re
import csv
import codecs
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation

from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers
import tensorflow as tf

In [ ]:
# Hyperparameters
MAX_SEQUENCE_LENGTH = 160
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.1

In [ ]:
# Data conversion into train and validation set
from keras.utils.np_utils import to_categorical
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(sentences)

sequences_1 = tokenizer.texts_to_sequences(X_train)
# sequences_1 = tokenizer.texts_to_sequences(train_data['message'].values)
sequences_test = tokenizer.texts_to_sequences(X_test)


word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))
data_1 = pad_sequences(sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
# data_2 = pad_sequences(sequences_2, maxlen=MAX_SEQUENCE_LENGTH)
data_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(y_train))
# labels = to_categorical(np.asarray(train_data['label'].values))
labels_test = to_categorical(np.asarray(y_test))
print('Shape of data tensor:', data_1.shape)
print('Shape of label tensor:', labels.shape)
indices = np.arange(data_1.shape[0])
np.random.shuffle(indices)
data = data_1[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

Found 104433 unique tokens
Shape of data tensor: (919305, 160)
Shape of label tensor: (919305, 2)


In [ ]:
print('Traing and validation set number of positive and negative reviews')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

Traing and validation set number of positive and negative reviews
[382674. 444701.]
[42483. 49447.]


##Embedding matrix

In [ ]:
print('Preparing embedding matrix')
nb_words = min(MAX_NB_WORDS, len(word_index))+1

embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in ft_sg_model.wv.vocab:
        embedding_matrix[i] = ft_sg_model.wv.word_vec(word)
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing embedding matrix
Null word embeddings: 70088


## Model Training 

In [ ]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_lstm = Bidirectional(LSTM(100))(embedded_sequences)
preds = Dense(2, activation='softmax')(l_lstm)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - Bidirectional LSTM")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=2, batch_size=32)

model fitting - Bidirectional LSTM
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 160)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 160, 100)          10443400  
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               160800    
_________________________________________________________________
dense (Dense)                (None, 2)                 402       
Total params: 10,604,602
Trainable params: 10,604,602
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
25856/25856 [==============================] - 7573s 293ms/step - loss: 0.6830 - acc: 0.5588 - val_loss: 0.6818 - val_acc: 0.5597
Epoch 2/2
25856/25856 [==============================] - 7726s 29

##Attention layer

In [ ]:
# Attention GRU network		  
class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializers.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros") 
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)



sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_gru = Bidirectional(GRU(100, return_sequences=True))(embedded_sequences)
l_att = AttLayer()(l_gru)
preds = Dense(2, activation='softmax')(l_att)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - attention GRU network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=2, batch_size=32)

model fitting - attention GRU network
Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 160)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 160, 100)          10443400  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 160, 200)          121200    
_________________________________________________________________
att_layer (AttLayer)         (None, 200)               360       
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402       
Total params: 10,565,362
Trainable params: 10,565,362
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
25856/25856 [==============================] - 7474s 289ms/s

##Prediction and Classification report

In [ ]:
from sklearn.metrics import classification_report
print('Start making the submission before fine-tuning')

preds = model.predict(data_test, batch_size=32, verbose=1)
pred_flat = np.argmax(preds, axis=1).flatten()
print(pred_flat)
print(classification_report(y_test, pred_flat))


Start making the submission before fine-tuning
3193/3193 [==============================] - 178s 56ms/step
[1 0 1 ... 1 1 1]
              precision    recall  f1-score   support

           0       0.56      0.32      0.41     47399
           1       0.57      0.78      0.66     54746

    accuracy                           0.57    102145
   macro avg       0.57      0.55      0.54    102145
weighted avg       0.57      0.57      0.54    102145

